In [1]:
import os 
import sys
import pandas as pd
import numpy as np
import torch

import torch.nn as nn
import torch.nn.functional as F

# Week2_2 Class


## 1. Pytorch Graph
`torcn.nn` 모듈은 텐서 그래프를 생성하는 다양한 함수를 제공한다.

[OFFICAL DOCUMENT](https://pytorch.org/docs/stable/nn.html)

## Table of Contents
1. [Container](#Container)
2. [Layers](#Layers)
3. [Loss](#Loss)
4. [추가](#To-Learn-More..)



### 1.1. Container
- [OFFICIAL DOC](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#module)

#### 1.1.1 `Module` class
- Neural Network를 생성할 때 반드시 `Module` 클래스를 부모 클래스로 상속받아야 함
- `Modul` 부모 클래스의 변수 및 메소드 사용 가능 (ex. `eval()`,`train()`, `parameters()`, `state_dict()`, `to()`)
- `forward()` 메소드는 모든 자식 클래스에서 반드시 **오버라이딩**해야 함
- [출처](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module)

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week2/week2_2_model.jpeg?raw=true" alt="model" width=600>

In [2]:
class Model(nn.Module):  # 반드시 Module class를 상속받아야 한다.
    def __init__(self, input_shape):
        super(Model, self).__init__()
        self.layer1= nn.Linear(input_shape, 32)
        self.layer2= nn.Linear(32, 64)
        self.layer_out= nn.Linear(64, 1)

        self.relu= nn.ReLU()

    def forward(self, x):
        x= self.relu(self.layer1(x))
        x= self.relu(self.layer2(x))
        x= self.layer_out(x)
        return x

In [3]:
model= Model(30) # train의 unit이 30

for param in model.parameters():
    print(param.shape)

for name, state in model.state_dict().items():
    print(f"{name} -> size: {state.shape}")

torch.Size([32, 30])
torch.Size([32])
torch.Size([64, 32])
torch.Size([64])
torch.Size([1, 64])
torch.Size([1])
layer1.weight -> size: torch.Size([32, 30])
layer1.bias -> size: torch.Size([32])
layer2.weight -> size: torch.Size([64, 32])
layer2.bias -> size: torch.Size([64])
layer_out.weight -> size: torch.Size([1, 64])
layer_out.bias -> size: torch.Size([1])


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
model.to(device)

cuda


Model(
  (layer1): Linear(in_features=30, out_features=32, bias=True)
  (layer2): Linear(in_features=32, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

#### 1.1.1 `Sequential` class
- 여러 layer를 연결한 container
- 이전 layer의 output이 다음 layer의 input으로 입력됨 (순차적)
- [출처](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html#torch.nn.Sequential)

In [5]:
model= nn.Sequential(
    nn.Linear(30, 32), # 1번 layer의 출력값이
    nn.ReLU(),         # 1번 activation 함수의 입력값으로 들어가고
    nn.Linear(32, 64), # 2번 layer의 입력값으로 1번 activation 함수의 출력값이 들어가서
    nn.ReLU(),         # 2번 activation의 입력값으로 입력
    nn.Linear(64,1)    # 그 결과가 다시 출력 layer로 입력되어 결과값 반환
)

model.eval()

Sequential(
  (0): Linear(in_features=30, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=1, bias=True)
)

### 1.2. Layers
- Linear( )
    - `input @ weight.T + bias`
- LSTM( )
    - [OFFICAL DOCS](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM) 
    - nn.LSTM(`input_size`, `hidden_size`)
    - `input` shape: (문장 길이, 배치 사이즈, 단어 임베딩 사이즈 == input suze)
    - `hidden_size` shape: (lstm 개수 * 레이어 수, 배치 사이즈, 히든 사이즈 == hidden size)

    <img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week2/week2_2_lstm.png?raw=true" width=500>
    - [출처](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

In [6]:
model= nn.Linear(20, 30)
print(f"W shape: {model.weight.shape}")
print(f"bias shape: {model.bias.shape}")

W shape: torch.Size([30, 20])
bias shape: torch.Size([30])


In [7]:
ex= "I love coding . Just kidding ."
inputs= ex.split()
print(inputs)

['I', 'love', 'coding', '.', 'Just', 'kidding', '.']


In [8]:
input_embedding= [torch.randn(1, 5) for _ in range(len(inputs))] # 임의로 난수로 단어 임베딩
print(f'input_embedding: {input_embedding}')
lstm= nn.LSTM(5, 5)  # (input dim, output dim)
hidden= (
    torch.randn(1, 1, 5),  # (모든 레이어의 lstm 개수, batch size, hidden_size)
    torch.randn(1, 1, 5),
)

# 한 단어씩 입력
for idx, i in enumerate(input_embedding):
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    print(f"{idx+1} word: output shape ({out.shape}) / hidden state shape ({hidden[0].shape})")

assert out.detach().equal(hidden[0].detach())

print('-------------------------------------------------------------------------------------------')

# sequence를 입력
input_embedding= torch.cat(input_embedding).view(len(inputs), 1, -1)
print(f"input sequence shape: {input_embedding.shape}")

hidden= (
    torch.randn(1, 1, 5),
    torch.randn(1, 1, 5),
)

out, hidden= lstm(input_embedding, hidden)
print(f"output shape: {out.shape}")
print(f"hidden shape: {hidden[0].shape}")

assert out[-1, :, :].detach().equal(hidden[0][-1,:,:].detach())

input_embedding: [tensor([[ 1.1381,  0.2423,  0.8096, -0.2866, -0.2022]]), tensor([[ 0.2569,  0.6500,  0.4125, -1.0496, -0.8965]]), tensor([[ 0.7071,  1.4113, -1.4082, -1.4036,  0.3996]]), tensor([[-0.9132, -0.0932, -0.1082,  1.0174, -1.0952]]), tensor([[ 0.1479, -1.5313, -0.0890, -0.7902,  0.6156]]), tensor([[-1.0946,  0.4375,  0.2711, -0.4052, -0.7780]]), tensor([[-1.1408,  0.1480,  0.3153,  0.7641, -1.0154]])]
1 word: output shape (torch.Size([1, 1, 5])) / hidden state shape (torch.Size([1, 1, 5]))
2 word: output shape (torch.Size([1, 1, 5])) / hidden state shape (torch.Size([1, 1, 5]))
3 word: output shape (torch.Size([1, 1, 5])) / hidden state shape (torch.Size([1, 1, 5]))
4 word: output shape (torch.Size([1, 1, 5])) / hidden state shape (torch.Size([1, 1, 5]))
5 word: output shape (torch.Size([1, 1, 5])) / hidden state shape (torch.Size([1, 1, 5]))
6 word: output shape (torch.Size([1, 1, 5])) / hidden state shape (torch.Size([1, 1, 5]))
7 word: output shape (torch.Size([1, 1, 5])

### 1.3. Activation
- nonlinear activations

In [9]:
nn.LeakyReLU()
nn.ReLU()
nn.Sigmoid()
nn.GELU()
nn.Tanh()
nn.Softmax()

Softmax(dim=None)

### 1.4. Loss
- loss = loss_class()
    - loss(`y_hat`, `y`)
    - `loss().backward()`
- Mean Square Error Loss 
    - <img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week2/week2_2_mse.png?raw=true" width=200>
- Cross Entropy Loss 
    - <img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week2/week2_2_ce.png?raw=true" width=200>
- Binary Cross Entropy Loss 
    - <img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week2/week2_2_bce.png?raw=true" width=500>

In [10]:
# l2 distance loss
nn.MSELoss()

# cross entropy
## multi class
nn.CrossEntropyLoss()  # softmax 0

## binary class
nn.BCELoss()
nn.BCEWithLogitsLoss() # sigmoid 0 + BCELoss()

BCEWithLogitsLoss()

In [11]:
batch_size= 3
C= 5
logits= torch.randn(batch_size, C, requires_grad=True)
print(f"Logits: {logits}")

loss= nn.CrossEntropyLoss()

target= torch.empty(batch_size, dtype= torch.long).random_(C)
print(f"Target: {target}")

loss= loss(logits, target)
print(f"Loss: {loss}")

loss.backward()

Logits: tensor([[ 0.3367,  2.9557, -1.0266,  1.2258,  0.5213],
        [ 0.2764,  0.0517,  0.3147, -1.1077,  1.9244],
        [-1.0124, -0.9056,  0.7089,  0.0415, -0.4226]], requires_grad=True)
Target: tensor([0, 1, 3])
Loss: 2.2416300773620605


# Week2_2 Assignment

## [BASIC](#Basic) 
- "네이버 영화 감성 분류" 데이터를 불러와 `pandas` 라이브러리를 사용해 **전처리** 할 수 있다.
- 적은 데이터로도 높은 성능을 내기 위해, pre-trained `BERT` 모델 위에 1개의 hidden layer를 쌓아 **fine-tuning**할 수 있다.

## [CHALLENGE](#Challenge)
- 토큰화된 학습 데이터를 배치 단위로 갖는 **traindata iterator**를 구현할 수 있다. 

## [ADVANCED](#Advanced)
- **loss와 optimizer 함수**를 사용할 수 있다. 
- traindata iterator를 for loop 돌며 **fine-tuning** 할 수 있다.
- fine-tuning의 2가지 방법론을 비교할 수 있다. 
  - BERT 파라미터를 **freeze** 한 채 fine-tuning (Vision에서 주로 사용하는 방법론)
  - BERT 파라미터를 **unfreeze** 한 채 fine-tuning (NLP에서 주로 사용하는 방법론)


### Reference
- [huggingface 한국어 오픈소스 모델](https://huggingface.co/models?language=ko&sort=downloads&search=bert)
- [transformer BertForSequenceClassification 소스 코드](https://github.com/huggingface/transformers/blob/v4.15.0/src/transformers/models/bert/modeling_bert.py#L1501)

In [12]:
import os
import sys
import pandas as pd
import numpy as np 
import torch
import random

In [13]:
# seed
seed = 7777
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [14]:
# device type
if torch.cuda.is_available():
    device= torch.device("cuda")
    print(f"# available GPUs: {torch.cuda.device_count()}")
    print(f"GPU name: {torch.cuda.get_device_name()}")
else:
    device= torch.device("cpu")

print(device)

# available GPUs: 1
GPU name: Tesla P100-PCIE-16GB
cuda


## Basic

### 데이터 다운로드 및 DataFrame 형태로 불러오기
- 내 구글 드라이브에 데이터를 다운받은 후 코랩에 드라이브를 마운트하면 데이터를 영구적으로 사용할 수 있음.
- [네이버영화감성분류](https://github.com/e9t/nsmc)
  - trainset: 150,000 
  - testset: 50,000 

In [15]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# 데이터 다운로드
!git clone https://github.com/e9t/nsmc.git

fatal: destination path 'nsmc' already exists and is not an empty directory.


In [17]:
!pip install transformers

In [18]:
_CUR_DIR = os.path.abspath(os.curdir)
print(f"My current directory : {_CUR_DIR}")
_DATA_DIR = os.path.join(_CUR_DIR, "nsmc")

My current directory : /content


In [19]:
# nsmc/ratings_train.txt를 DataFrame 형태로 불러오기
df = pd.read_csv('/content/nsmc/ratings_train.txt', sep='\t')
print(df.shape)
df.head()

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


### 데이터 결측치 제거 및 데이터 수 줄이기 
- 학습 데이터 수는 150,000개로 매우 많은 양이다. 하지만 우리가 실생활에서 마주할 데이터는 이렇게 많지 않다. 이 때 유용하게 사용되는 것이 **fine-tuning** 학습 방법이다.   
- Fine-tuning은 단어의 의미를 이미 충분히 학습한 모델 (여기서는 **BERT**)을 가져와 그 위에 추가적인 Nueral Network 레이어를 쌓은 후 학습하는 방법론이다. 이미 BERT가 단어의 의미를 충분히 학습했기 때문에 **적은 데이터**로 학습해도 우수한 성능을 낼 수 있다는 장점이 있다. 
- **데이터의 label의 비율이 5:5를 유지하면서** 학습 데이터 수를 150,000개에서 1,000개로 줄이는 함수 `label_evenly_balanced_dataset_sampler`를 구현하라.
  - 함수 정의 
    - 입력 매개변수
      - df : DataFrame
      - n_sample : df에서 샘플링할 row의 개수 (여기서는 1000개로 정의한다)
    - 조건
      - label의 비율이 5:5를 유지할 수 있도록 샘플링한다.
    - 반환값
      - row의 개수가 1000개인 dataframe

In [20]:
# df에서 결측치 (na 값) 제거

df = df.dropna()
df.shape

(149995, 3)

In [21]:
# label별 데이터 수 확인
# pandas의 value_counts 함수 활용
# 0 -> 부정 1 -> 긍정

df.label.value_counts()

0    75170
1    74825
Name: label, dtype: int64

In [22]:
# 학습 데이터 샘플 개수 설정

n_sample = 1000

# 샘플링 함수 구현
# random 모듈에서 제공되는 함수 활용
# input: 학습 데이터 샘플 개수
# output: 샘플링 데이터


def label_evenly_balanced_dataset_sampler(df, sample_size):
  """
  데이터 프레임의을 sample_size만큼 임의 추출해 새로운 데이터 프레임을 생성.
  이 때, "label"열의 값들이 동일한 비율을 갖도록(5:5) 할 것.
  """
  num_sample= int(sample_size / 2)
  label_0 = df[df['label']==0].sample(n=num_sample)
  label_1 = df[df['label']==1].sample(n=num_sample)

  sample = pd.concat([label_0, label_1])


  return sample

sample_df = label_evenly_balanced_dataset_sampler(df, n_sample)

In [23]:
# 검증

sample_df.label.value_counts()

0    500
1    500
Name: label, dtype: int64

### CustomClassifier 클래스 구현
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week2/week2_2_bertclf.png?raw=true" width=400>

- 그림과 같이 사전 학습(pre-trained)된 `BERT` 모델을 불러와 그 위에 **1 hidden layer**와 **binary classifier layer**를 쌓아 fine-tunning 모델을 생성할 것이다.    
---
- hidden layer 1개와 output layer(binary classifier layer)를 갖는 `CustomClassifier` 클래스를 구현하라.
- 클래스 정의
  - 생성자 입력 매개변수
    - `hidden_size` : BERT의 embedding size
    - `n_label` : class(label) 개수
  - 생성자에서 생성할 변수
    - `bert` : BERT 모델 인스턴스 
    - `classifier` : 1 hidden layer + relu +  dropout + classifier layer를 stack한 `nn.Sequential` 모델
      - 첫번재 히든 레이어 (첫번째 `nn.Linear`)
        - input: BERT의 마지막 layer의 1번재 token ([CLS] 토큰) (shape: `hidden_size`)
        - output: (shape: `linear_layer_hidden_size`)
      - 아웃풋 레이어 (두번째 `nn.Linear`)
        - input: 첫번째 히든 레이어의 아웃풋 (shape: `linear_layer_hidden_size`)
        - output: target/label의 개수 (shape:2)
  - 메소드
    - `forward()`
      - BERT output에서 마지막 레이어의 첫번째 토큰 ('[CLS]')의 embedding을 가져와 `self.classifier`에 입력해 아웃풋으로 logits를 출력함.
  - 주의 사항
    - `CustomClassifier` 클래스는 부모 클래스로 `nn.Module`을 상속 받는다.


In [24]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel

In [25]:
# classifier 구현
class CustomClassifier(nn.Module):

  def __init__(self, hidden_size: int, n_label: int):
    super(CustomClassifier, self).__init__()

    self.bert = BertModel.from_pretrained("klue/bert-base")

    dropout_rate = 0.1
    linear_layer_hidden_size = 32

    self.classifier = nn.Sequential(
                            nn.Linear(hidden_size,linear_layer_hidden_size),
                            nn.ReLU(),
                            nn.Dropout(dropout_rate),
                            nn.Linear(linear_layer_hidden_size,n_label)
                                    ) # torch.nn에서 제공되는 Sequential, Linear, ReLU, Dropout 함수 활용


  def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):

    outputs = self.bert(
        input_ids,
        attention_mask=attention_mask,
        token_type_ids=token_type_ids,
    )

    # BERT 모델의 마지막 레이어의 첫번재 토큰을 인덱싱
    last_hidden_state= outputs[0]
    cls_token_last_hidden_states = last_hidden_state[:,0,:] # 마지막 layer의 첫 번째 토큰 ("[CLS]") 벡터를 가져오기, shape = (1, hidden_size)

    logits = self.classifier(cls_token_last_hidden_states)

    return logits

## Challenge

### 학습 데이터를 배치 단위로 저장하는 이터레이터 함수 `data_iterator` 구현
- 데이터 프레임을 입력 받아 text를 토큰 id로 변환하고 label은 텐서로 변환해 배치만큼 잘라 (input, target) 튜플 형태의 이터레이터를 생성하는 `data_iterator` 함수를 구현하라.
- 함수 정의 
  - 입력 매개변수
    - `input_column` : text 데이터 column 명
    - `target_column` : label 데이터 column 명
    -  `batch_size` : 배치 사이즈
  - 조건
    - 함수는 다음을 수행해야 함 
      - 데이터 프레임 랜덤 셔플링
      - `tokenizer_bert`로 text를 token_id로 변환 + 텐서화 
      - target(label)을 텐서화
  - 반환값 
    - (input, target) 튜플 형태의 이터레이터를 반환

In [26]:
from transformers import BertTokenizer, BertModel
tokenizer_bert= BertTokenizer.from_pretrained('klue/bert-base')

#### Tokenizing 예시(1개의 문장)

In [27]:
# 1. string type의 문장을 가져옴
ex_sent= sample_df.document.iloc[0]
print(f"Original Sentence: {ex_sent}\n")

# 2. 문장을 토크나이즈함. 이때, 특수 토큰 ("[CLS]", "[SEP]")을 자동으로 추가하고 pytorch의 tensor 형태로 변환해 반환함
tensor_sent= tokenizer_bert(
    ex_sent, 
    add_special_tokens=True, # 문장의 앞에 문장 시작을 알리는 "[CLS]" 토큰, 문장의 마지막에 문장 끝을 알리는 "[SEP]" 토큰을 추가함
    return_tensors='pt' # pytorch tensor로 반환할 것
)

print(f"Tokenized Sentence: \n{tensor_sent}")

Original Sentence: 도대체 뭐냐..이건 ㅋㅋ

Tokenized Sentence: 
{'input_ids': tensor([[   2, 6641, 1097, 2529,   18,   18, 5370, 3725,    3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}


#### Tokenizing 예시(2개의 문장)

In [28]:
# 1. 2개의 문장을 가진 list 생성
ex_sent_list= list(sample_df.document.iloc[:2].values)
for i, sent in enumerate(ex_sent_list):
    print(f"Original Sentence {i+1}: {sent}")

# 2. 문장 리스트를 토크나이즈 함. 
# 이때, 리스트 내 문장들의 토큰 길이가 동일할 수 있도록 가장 긴 문장을 기준으로 부족한 위치에 "[PAD]" 토큰을 추가함
tensor_sent_list= tokenizer_bert(
    ex_sent_list,
    add_special_tokens=True,
    return_tensors='pt',
    padding='longest'  # 가장 긴 문장을 기준으로 token 개수를 맞춤. 모자른 토큰 위치는 "[PAD]" 토큰을 추가
)

print(f"\nTokenized Sentence list: {tensor_sent_list}")

# 토크나이즈된 두 문장의 길이가 동일함을 검증
assert tensor_sent_list['input_ids'][0].shape == tensor_sent_list['input_ids'][1].shape

Original Sentence 1: 도대체 뭐냐..이건 ㅋㅋ
Original Sentence 2: 너무 어중간함. 스파이 영화라기엔 액션이 너무 없고, 연애물이라기엔 너무 밋밋하고. 이런 영화는 양쪽의 장점을 살려야 하는데 둘다 망함. 그 결과 볼거리가 없고 지루함

Tokenized Sentence list: {'input_ids': tensor([[    2,  6641,  1097,  2529,    18,    18,  5370,  3725,     3,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [    2,  3760,  1406, 22227,  2530,    18, 13106,  3771, 16947,  2614,
          8765,  2052,  3760,  1415,  2088,    16,  7738,  2266,  2052, 16947,
          2614,  3760, 23977, 19521,    18,  3667,  3771,  2259,  8108,  2079,
          5472,  2069, 28049,  1889, 13964,   867,  2062,  1046,  2530,    18,
           636,  3731, 10421,  2116,  1415,  2088,  9734,  2530,     3]]), 'token_type_ids': tensor([[0, 

In [39]:
def data_iterator(df, input_column, target_column, batch_size):
  """
  데이터 프레임을 셔플한 후 
  데이터 프레임의 input_column을 batch_size만큼 잘라 토크나이즈 + 텐서화하고, target_column을 batch_size만큼 잘라 텐서화 하여
  (input, output) 튜플 형태의 이터레이터를 생성
  """

  global tokenizer_bert

  # 1. 데이터 프레임 셔플
  #    pandas의 sample 함수 사용
  df = df.sample(frac=1, random_state=seed).reset_index(drop=True)

  # 2. 이터레이터 생성
  for idx in range(0, df.shape[0], batch_size):
    batch_df = df.iloc[idx:idx+batch_size] # batch_size만큼 데이터 추출
    
    tensorized_input = tokenizer_bert(
                            list(batch_df[input_column].values),
                            add_special_tokens=True,
                            return_tensors='pt',
                            padding='longest'
    ) # df의 text를 토크나이징 + token id로 변환 + 텐서화 (df의 input_column 사용)
    
    tensorized_target = torch.tensor(list(batch_df[target_column].values)) # target(label)을 텐서화 (df의 target_column 사용)

    yield tensorized_input, tensorized_target # 튜플 형태로 yield

In [40]:
batch_size=32
train_iterator = data_iterator(sample_df, 'document', 'label', batch_size)

In [41]:
next(train_iterator)

({'input_ids': tensor([[    2, 20225,  2961,  ...,     0,     0,     0],
         [    2, 10499,  2116,  ...,     0,     0,     0],
         [    2,  1556,  4390,  ...,     0,     0,     0],
         ...,
         [    2,  3919,  2119,  ...,     0,     0,     0],
         [    2, 17211,  2119,  ...,     0,     0,     0],
         [    2, 11840,  2410,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])},
 tensor([0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
         1, 0, 1, 0, 1, 1, 0, 0]))

## Advanced

### `data_iterator` 함수로 생성한 이터레이터를 for loop 돌면서 배치 단위의 데이터를 모델에 학습하는 `train()` 함수 구현
- 함수 정의
  - 입력 매개변수
    - `model` : BERT + 1 hidden layer classifier 모델
    - `data_iterator` : train data iterator
- Reference
  - [Loss: CrossEntropyLoss official document](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
  - [Optimizer: AdamW official document](https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html)

In [42]:
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from numpy.core.fromnumeric import nonzero

In [43]:
# 모델 클래스 정의
model= CustomClassifier(hidden_size=768, n_label=2)

batch_size= 32

# 데이터 이터레이터 정의
train_iterator= data_iterator(sample_df, 'document', 'label', batch_size)

# 로스 및 옵티마이저
loss_fct= CrossEntropyLoss()
optimizer= AdamW(
    model.parameters(),
    lr= 2e-5,
    eps= 1e-8
)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [44]:
def train(model, data_iterator):

    global loss_fct # 위에서 정의한 loss 함수

    # 배치 단위 평균 loss와 총 평균 loss를 계산하기 위해 변수 생성
    total_loss, batch_loss, batch_count= 0, 0, 0

    # model을 train 모드로 설정 & device 할당
    model.train()
    model.to(device)

    # data iterator를 돌면서 하나씩 학습
    for step, batch in enumerate(data_iterator):
        batch_count+=1

        # tensor 연산 전, 각 tensor에 device 할당
        batch= tuple(item.to(device) for item in batch)

        batch_input, batch_label= batch

        # batch 마다 모델이 갖고 있는 기존 gradient를 초기화
        model.zero_grad()

        # forward
        logits= model(**batch_input)

        # loss
        loss= loss_fct(logits,batch_label)
        batch_loss+= loss.item()
        total_loss+= loss.item()

        # backward -> 파라미터의 미분(gradient)를 자동으로 계산
        loss.backward()

        # optimizer 업데이트
        optimizer.step()

        # 배치 10개씩 처리할 때마다 평균 loss를 출력
        if (step % 10 == 0 and step != 0):
            print(f"Step: {step}, Avg Loss: {batch_loss / batch_count:.4f}")

            # 변수 초기화
            batch_loss, batch_count= 0,0

    print(f"Mean Loss : {total_loss/(step+1):.4f}")
    print("Train Finished")

### 지금까지 구현한 함수와 클래스를 모두 불러와 `train()` 함수를 실행하자
- fine-tuning 모델 클래스 (`CustomClassifier`)
    - hidden_size = 768
    - n_label = 2
- 데이터 이터레이터 함수 (`data_iterator`)
    - batch_size = 32
- loss 
    - `CrossEntropyLoss()`
- optimizer
    - optimizer는 loss(오차)를 상쇄하기 위해 파라미터를 업데이트 하는 과정
    - `optimizer.step()` 시 파라미터가 업데이트 됨 
    - [Optimizer 종류 설명](https://ganghee-lee.tistory.com/24)
    - `AdamW()`
        - [AdamW official document](https://pytorch.org/docs/1.9.1/generated/torch.optim.AdamW.html?highlight=adamw)
    - lr = 2e-5
    

In [45]:
# 모델 
model= CustomClassifier(hidden_size=768, n_label=2)

# 데이터 이터레이터
batch_size= 32
train_iterator= data_iterator(sample_df, 'document', 'label', batch_size)

# 로스 및 옵티마이저
loss_fct= CrossEntropyLoss()
optimizer= AdamW(
    model.parameters(),
    lr= 2e-5,
    eps= 1e-8,
)

# 학습 시작
train(model, train_iterator)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Step: 10, Avg Loss: 0.6483
Step: 20, Avg Loss: 0.5094
Step: 30, Avg Loss: 0.4625
Mean Loss : 0.5424
Train Finished


## fine-tuning 2가지 방법론 비교
- pre-trained BERT 모델 파라미터를 **freeze**한 채 학습하라
    - BERT의 파라미터의 `requires_grad`값을 `False`로 바꾸면, 학습 시 BERT의 파라미터는 미분이 계산되지도, 업데이트 되지도 않는다.
    - 이렇게 특정 모델의 파라미터가 업데이트 하지 못하도록 설정하는 것을 **freeze**라고 한다.
    - BERT 파라미터를 freeze시킨 채 학습을 진행해보자. 이럴 경우, 우리가 직접 쌓은 fine-tuning layer의 파라미터만 업데이트 된다.
- **unfreeze**와 **freeze**모델의 성능을 비교해보자. 어떤 방식이 더 우수한다?

In [61]:
class CustomClassifierFreezed(nn.Module):

    def __init__(self, hidden_size: int, n_label: int):
        super(CustomClassifierFreezed, self).__init__()

        self.bert= BertModel.from_pretrained('klue/bert-base')
        # freeze BERT parameter
        # BERT의 파라미터는 고정값으로 두고 BERT 위에 씌운 linear layer의 파라미터만 학습하려고 한다.
        # 이 경우, BERT의 파라미터의 'requires_grad' 값을 False로 변경해줘야 학습 시 해당 파라미터의 미분값이 계산되지 않는다.
        for param in self.bert.parameters():
            param.requires_grad= False

        dropout_rate= 0.1
        linear_layer_hidden_size= 32

        self.classifier= nn.Sequential(
                            nn.Linear(hidden_size,linear_layer_hidden_size),
                            nn.ReLU(),
                            nn.Dropout(dropout_rate),
                            nn.Linear(linear_layer_hidden_size, n_label)
                                    )

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):
        outputs= self.bert(
            input_ids,
            attention_mask= attention_mask,
            token_type_ids= token_type_ids,
        )

        # BERT 모델의 마지막 레이어의 첫번째 토큰을 인덱싱
        last_hidden_states= outputs[0]
        cls_token_last_hidden_states= last_hidden_states[:,0,:]
        logits= self.classifier(cls_token_last_hidden_states)

        return logits

In [62]:
# freeze 모델
# model을 제외한 설정값은 위에서 실행한 unfreeze 모델과 동일
model = CustomClassifierFreezed(hidden_size=768, n_label=2)

# 데이터 이터레이터
batch_size = 32
train_iterator = data_iterator(sample_df, 'document', 'label', batch_size)

# 로스 및 옵티마이저
loss_fct = CrossEntropyLoss()
optimizer = AdamW(
    model.parameters(),
    lr= 2e-5,
    eps= 1e-9
)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [63]:
# 학습 시작
train(model, train_iterator)

Step: 10, Avg Loss: 0.7250
Step: 20, Avg Loss: 0.7251
Step: 30, Avg Loss: 0.6858
Mean Loss : 0.7132
Train Finished
